In [ ]:
import subprocess
import io
import gzip
import os

import pandas as pd

from collections import OrderedDict
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC

# Generate the diamonds DBs

In [ ]:
fld = pd.read_csv("name_dictionary.csv", index_col=0, sep="\t", header=None)
fld = fld.to_dict()[1]
fld[list(fld.keys())[0]]

In [ ]:
BRHdb = pd.read_csv("BRHdb.csv", index_col=None, sep="\t", header=0)
# selected_list = BRHdb.names.unique()
# BRHdb = BRHdb[BRHdb["name"].isin([selected_list])]

In [ ]:
# generate a diamond db for every fasta.gz file
for fl in fld.keys():    
    os.system("gunzip --keep --stdout files/{0} | \
               diamond makedb --in /dev/stdin --db dbs/{1}".format(fld[fl], fl))

# Get the gene list from the reference fasta file

In [ ]:
def ref_gene_list(file_name):
    """
    file_name: name of fa.gz file containing the amino-acid sequences of all genes
               I ended up using the longest transcript files so might be simplified
    """
    fasta_file = gzip.open("files/"+file_name, mode='rt')
    record_iterator = SeqIO.parse(fasta_file, "fasta")
    od = OrderedDict()

    for record in record_iterator:
        record.id = str(record.id).split("_")[0]
        record.name = ""
        record.description = ""
        if record.id in od:
            if len(record.seq) > len(od[record.id].seq):
                od[record.id] = record
        else:
            od[record.id] = record

    len(od.keys())
    rbh_df = pd.DataFrame(index=od.keys())
    return(rbh_df)

In [ ]:
def generate_RBH(name, ref, qlist, fld):
    """
    name:  the name of the output RBHdb
    ref:   the name of the reference genome to be used 
    qlist: a list containing the query genomes to be used
    fld:   a dictionary converting genome names to file names
    """
    # An empty dataframe containing all the gene IDs in the reference genome
    rbh_df = ref_gene_list(fld[ref])
    
    # blast every non-reference fasta.gz file against the reference fasta.gz
    for fl in qlist:
        # Skip self-comparison if accidently included in the list
        if fl == ref:
            continue

        # /dev/stdout only works if provided with permission: sudo chown -R $USER /dev
        # Run Fwd comparison - ref vs. query
        fwd = pd.read_csv(io.StringIO(subprocess.check_output("gunzip --keep --stdout files/{0} | \
                   diamond blastp --quiet -p 32 -d dbs/{1} -q /dev/stdin -o /dev/stdout \
                   --more-sensitive --outfmt 6 qseqid sseqid bitscore".format(fld[ref], fl),
                   shell=True, text=True)), sep="\t", header=None)

        # Run Reciprocal Rev comparison - query vs ref    
        rev = pd.read_csv(io.StringIO(subprocess.check_output("gunzip --keep --stdout files/{0} | \
                   diamond blastp --quiet -p 32 -d dbs/{1} -q /dev/stdin -o /dev/stdout \
                   --more-sensitive --outfmt 6 qseqid sseqid bitscore".format(fld[fl], ref),
                   shell=True, text=True)), sep="\t", header=None)

        # Add headers to forward and reverse results dataframes
        headers = ["query", "subject", "bitscore"]
        fwd.columns = headers
        rev.columns = headers

        # https://widdowquinn.github.io/2018-03-06-ibioic/02-sequence_databases/05-blast_for_rbh.html
        # Merge forward and reverse results
        rbbh = pd.merge(fwd, rev[['query', 'subject']],
                        left_on='subject', right_on='query', how='outer')

        # Discard rows that are not RBH
        rbbh = rbbh.loc[rbbh.query_x == rbbh.subject_y]

        # Discard genes with multiple RBHs based, keeping highest biscore pairs
        rbbh = rbbh.sort_values(by="bitscore", axis=0, ascending=False) 
        rbbh = rbbh.drop_duplicates(subset=["subject_y"])
        
        # Drop all column except reference and query gene IDs
        rbbh = rbbh.drop(["bitscore", "query_y", "subject_y"], axis=1)
        rbbh.columns = [ref, fl]
        rbbh = rbbh.set_index(ref)

        # join the results with the main rbh_df by adding a new column
        rbh_df = rbh_df.join(rbbh, how='outer')
    
    # save the final RBH dataframe as a csv file
    rbh_df.to_csv("RBHs/RBH_"+name+".tsv", sep="\t")

In [ ]:
# Iterate over the BRHdb table to generate each individual RBHdb
for name in BRHdb["name"].unique():
    ref = BRHdb[BRHdb["name"].isin([name])]["reference"].unique()[0]
    qlist = BRHdb[BRHdb["name"].isin([name])]["query"].to_list()
    print(name, ref)
    print(qlist)
    generate_RBH(name, ref, qlist, fld)